<a href="https://colab.research.google.com/github/eutiagovski/amex-default-predicition/blob/main/pre-processing/ADC-DataPre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ACKNOWLEDGEMENTS

    This notebook was inspired on @susnato

    Because the originl data are too large to fit in memory, I am using the dataset of Feather & PArquet files by @RADDAR

    The Techniques I used to reduce the size of the data is inspired from this great discussion by @cdeotte

    Many of the techniques related to Numerical Features are inspired from this awesome notebook by @lucasmorin


#### imports

In [1]:
import os
import gc
import glob
import tqdm
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('max_rows', 100)
pd.set_option('max_columns', 300)

###loading data

The dataset contains aggregated profile features for each customer at each statement date. Features are anonymized and normalized, and fall into the following general categories:

    D_* = Delinquency variables
    S_* = Spend variables
    P_* = Payment variables
    B_* = Balance variables
    R_* = Risk variables

with the following features being categorical:

['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

Your task is to predict, for each customer_ID, the probability of a future payment default (target = 1).

In [2]:
train_set = pd.read_parquet("/content/drive/MyDrive/kaggle/amex-default-predict/train.parquet")
train_labels = pd.read_csv("/content/drive/MyDrive/kaggle/amex-default-predict/train_labels.csv")
test_set = pd.read_parquet("/content/drive/MyDrive/kaggle/amex-default-predict/test.parquet")
submission = pd.read_csv('/content/drive/MyDrive/kaggle/amex-default-predict/sample_submission.csv')

###features engine


In [3]:
### preprocessing data (https://www.kaggle.com/code/susnato/amex-data-preprocesing-feature-engineering)

print(train_set.shape, train_labels.shape)
print(test_set.shape, submission.shape)

bin_cols = ['B_31', 'D_87']
cat_cols = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
num_cols = list(set(train_set.columns)-set(cat_cols+['S_2', 'customer_ID']))

int8_num_cols = list(set(train_set.dtypes[train_set.dtypes==np.int8].axes[0]) - set(cat_cols))
int16_num_cols = list(set(train_set.dtypes[train_set.dtypes==np.int16].axes[0]) - set(cat_cols))
float32_num_cols = list(set(train_set.dtypes[train_set.dtypes==np.float32].axes[0]) - set(cat_cols))

def last_2(series):
    return series.values[-2] if len(series.values)>=2 else -127

def last_3(series):
    return series.values[-3] if len(series.values)>=3 else -127


print("We have {} Categorical features and {} Numerical features".format(len(cat_cols), len(num_cols)))


(5531451, 190) (458913, 2)
(11363762, 190) (924621, 2)
We have 11 Categorical features and 177 Numerical features


In [ ]:
### Encode customer ids(https://www.kaggle.com/competitions/amex-default-prediction/discussion/328054)

train_set['customer_ID'] = train_set['customer_ID'].apply(lambda x: int(x[-16:], 16)).astype(np.int64)
train_labels['customer_ID'] = train_labels['customer_ID'].apply(lambda x: int(x[-16:], 16)).astype(np.int64)
# test_set['customer_ID'] = test_set['customer_ID'].apply(lambda x: int(x[-16:], 16)).astype(np.int64)
# submission['customer_ID'] = submission['customer_ID'].apply(lambda x: int(x[-16:], 16)).astype(np.int64)

In [4]:
### features engine (https://www.kaggle.com/code/susnato/amex-data-preprocesing-feature-engineering)

def take_first_col(series):
    return series.values[0]

def prepare_date_features(df):
    ### Drop all other columns except the S_2 and customer_ID(cat_cols, num_cols)
    df = df.drop(cat_cols+num_cols, axis=1)
    
    ### Converting S_2 column to datetime column
    df['S_2'] = pd.to_datetime(df['S_2'])

    ### How many rows of records does each customer has?
    df['rec_len'] = df[['customer_ID', 'S_2']].groupby(by=['customer_ID'])['S_2'].transform('count').astype(np.int8)

    ### Encode the 1st statement and the last statement time
    df['S_2_first'] = df[['customer_ID', 'S_2']].groupby(by=['customer_ID'])['S_2'].transform('min')
    df['S_2_last'] = df[['customer_ID', 'S_2']].groupby(by=['customer_ID'])['S_2'].transform('max')

    ### For how long(days) the customer is receiving the statements
    df['S_2_period'] = (df[['customer_ID', 'S_2']].groupby(by=['customer_ID'])['S_2'].transform('max') - df[['customer_ID', 'S_2']].groupby(by=['customer_ID'])['S_2'].transform('min')).dt.days.astype(np.float16)

    ### Days Between 2 statements 
    df['days_between_statements'] = df[['customer_ID', 'S_2']].sort_values(by=['customer_ID', 'S_2']).groupby(by=['customer_ID'])['S_2'].transform('diff').dt.days.astype(np.float16)
    df['days_between_statements'] = df['days_between_statements'].fillna(0)
    df['days_between_statements_mean'] = df[['customer_ID', 'days_between_statements']].sort_values(by=['customer_ID', 'days_between_statements']).groupby(by=['customer_ID']).transform('mean').astype(np.float16)
    df['days_between_statements_std'] = df[['customer_ID', 'days_between_statements']].sort_values(by=['customer_ID', 'days_between_statements']).groupby(by=['customer_ID']).transform('std').fillna( df['days_between_statements_mean']).astype(np.float16)
    df['days_between_statements_max'] = df[['customer_ID', 'days_between_statements']].sort_values(by=['customer_ID', 'days_between_statements']).groupby(by=['customer_ID']).transform('max').astype(np.float16)
    df['days_between_statements_min'] = df[['customer_ID', 'days_between_statements']].sort_values(by=['customer_ID', 'days_between_statements']).groupby(by=['customer_ID']).transform('min').astype(np.float16)

    ### https://www.kaggle.com/code/lucasmorin/amex-lgbm-features-eng/notebook
    df['S_2'] = (df['S_2_last']-df['S_2']).dt.days.astype(np.float16)

    ### Difference between S_2_last(max) and S_2_last 
    df['S_2_last_diff'] = (df['S_2_last'].max()-df['S_2_last']).dt.days.astype(np.float16)

    ### Difference between S_2_first(min) and S_2_first 
    df['S_2_first_diff'] = (df['S_2_first'].min()-df['S_2_first']).dt.days.astype(np.float16)

    ### Get the (day,month,year) and drop the S_2_first because we can't directly use them
    df['S_2_first_dd'] = df['S_2_first'].dt.day.astype(np.int8)
    df['S_2_first_mm'] = df['S_2_first'].dt.month.astype(np.int8)
    df['S_2_first_yy'] = df['S_2_first'].dt.year.astype(np.int8)
    
    df['S_2_last_dd'] = df['S_2_last'].dt.day.astype(np.int8)
    df['S_2_last_mm'] = df['S_2_last'].dt.month.astype(np.int8)
    df['S_2_last_yy'] = df['S_2_last'].dt.year.astype(np.int8)
    
    agg_df = df.groupby(by=['customer_ID']).agg({'S_2':['last', last_2, last_3],
                                                 'days_between_statements':['last', last_2, last_3]})
    agg_df.columns = [i+'_'+j for i in ['S_2', 'days_between_statements'] for j in ['last', 'last_2', 'last_3']]
    df = df.groupby(by=['customer_ID']).agg(take_first_col)
    df = df.merge(agg_df, how='inner', left_index=True, right_index=True)
    df = df.drop(['S_2', 'days_between_statements', 'S_2_first', 'S_2_last_x'], axis=1)

    return df 


In [5]:
def prepare_numerical_features(df):
    for num_c in list(num_cols):
        col_dtype = df[num_c].dtype
        df[num_c] = df[num_c].fillna(df[num_c].mean())
        df[num_c] = df[num_c].astype(col_dtype)
    
    df['S_2'] = pd.to_datetime(df['S_2'])
    df = df.sort_values(by=['customer_ID', 'S_2'])
    ### Drop cat columns and S_2 so that you only have num features and customer_ID
    df = df.drop(cat_cols+['S_2'], axis=1)
    num_feature_list = ['min', 'max', 'mean', 'std', 'last', last_2, last_3]
    
    df_float32_agg = df[['customer_ID']+float32_num_cols].groupby(by=['customer_ID']).agg(num_feature_list).astype(np.float32)
    df_float32_agg.columns = [str(c[0])+'_'+str(c[1]) for c in df_float32_agg.columns]
    
    df_int_agg = df[['customer_ID']+int8_num_cols+int16_num_cols].groupby(by=['customer_ID']).agg(num_feature_list).astype(np.float16)
    df_int_agg.columns = [str(c[0])+'_'+str(c[1]) for c in df_int_agg.columns]
    
    #df_agg = df.groupby(by=['customer_ID']).agg(num_feature_list).astype(np.float32)
    #df_agg.columns = [str(c[0])+'_'+str(c[1]) for c in df_agg.columns]
    df_agg = df_float32_agg.merge(df_int_agg, left_index=True, right_index=True)
    df_agg[[ii+'_last' for ii in int8_num_cols]] = df_agg[[ii+'_last' for ii in int8_num_cols]].astype(np.int8)
    df_agg[[ii+'_last_2' for ii in int8_num_cols]] = df_agg[[ii+'_last_2' for ii in int8_num_cols]].astype(np.int8)
    df_agg[[ii+'_last_3' for ii in int8_num_cols]] = df_agg[[ii+'_last_3' for ii in int8_num_cols]].astype(np.int8)
    
    del df, df_float32_agg, df_int_agg
    gc.collect()
    return df_agg



In [6]:
def prepare_cat_features(df):
    remove = ['customer_ID']

    agg_dict_num = {}
    agg_dict_cat = {}

    mean_diff = lambda x: np.nanmean(np.diff(x.values))
    mean_diff.__name__ = 'mean_diff'

    for c in df.columns:
        if c not in remove:
            if c not in cat_cols+bin_cols:
                agg_dict_num[c] = ['mean','std','min','max','last', last_2, last_3]
            else:
                agg_dict_cat[c] = ['nunique', ] 
    
    df.loc[:,cat_cols+bin_cols] = df.loc[:,cat_cols+bin_cols].astype(str)
    df_agg = df.groupby('customer_ID').agg(agg_dict_cat)
    df_agg.columns = [str(c[0])+'_'+str(c[1]) for c in df_agg.columns]
    df_list = []
    for c in cat_cols+bin_cols:
        df_cat = df.groupby(['customer_ID',c])[c].count()
        df_cat = df_cat.unstack()
        df_cat.columns = [df_cat.columns.name + '_' + c for c in df_cat.columns]
        df_cat = df_cat.fillna(0)
        df_list.append(df_cat)
    df_out = pd.concat([df_agg]+df_list, axis=1)
    df_out = df_out.fillna(np.nanmean(df_out))
    
    del df
    gc.collect()
    return df_out



In [ ]:
from sklearn.model_selection import StratifiedKFold

skfolds = StratifiedKFold(n_splits=6)
train_5_fold_splits = []

for train_index, test_index in skfolds.split(train_labels['customer_ID'], train_labels['target']):
  print(train_labels.iloc[test_index]['target'].value_counts())
  train_5_fold_splits.append(train_labels.iloc[test_index]['customer_ID'])

train_labels = train_labels.set_axis(train_labels['customer_ID'])
train_labels = train_labels.drop(['customer_ID'], axis=1)

0    85022
1    29707
Name: target, dtype: int64
0    85021
1    29707
Name: target, dtype: int64
0    85021
1    29707
Name: target, dtype: int64
0    85021
1    29707
Name: target, dtype: int64


In [ ]:
# split train

for (i,ids) in enumerate(train_5_fold_splits):
    # if i == 1:
    #   break
    print(i, len(ids))
    train_data_part = train_set[train_set.customer_ID.isin(ids)].sort_values(by=['customer_ID'])
    train_data_time = prepare_date_features(train_data_part).sort_values(by=['customer_ID'])
    train_data_num = prepare_numerical_features(train_data_part).sort_values(by=['customer_ID'])
    train_data_cat = prepare_cat_features(train_data_part).sort_values(by=['customer_ID'])
    
    assert list(train_data_time.axes[0])==list(train_data_num.axes[0])==list(train_data_cat.axes[0])

    y = train_labels.loc[ids]['target']
    x = train_data_time.merge(train_data_cat, left_index=True, right_index=True).merge(train_data_num, left_index=True, right_index=True)
    
    ### Save to Pickle
    train_data_time.merge(train_data_cat, left_index=True, right_index=True).merge(train_data_num, left_index=True, right_index=True).to_pickle('/content/drive/MyDrive/kaggle/amex-default-predict/train_data_{}.pkl'.format(i))
    np.save("/content/drive/MyDrive/kaggle/amex-default-predict/train_y_{}.npy".format(i), y)

    # callback_to_my_function

    del train_data_time, train_data_num, train_data_cat, train_data_part, y
    gc.collect()


0 114729
1 114728
2 114728
3 114728


In [ ]:
# split test
# https://stackoverflow.com/questions/2130016/splitting-a-list-into-n-parts-of-approximately-equal-length

def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))

test_split_ids = split(test_set.customer_ID.unique(),12)

for (i,ids) in enumerate(test_split_ids):
    print(i, len(ids))
    test_data_part = test_set[test_set.customer_ID.isin(ids)].sort_values(by=['customer_ID'])
    
    test_data_time = prepare_date_features(test_data_part).sort_values(by=['customer_ID'])
    test_data_num = prepare_numerical_features(test_data_part).sort_values(by=['customer_ID'])
    test_data_cat = prepare_cat_features(test_data_part).sort_values(by=['customer_ID'])
    assert list(test_data_part.axes[0])==list(test_data_part.axes[0])==list(test_data_part.axes[0])
    ### Save to Pickle
    test_data_time.merge(test_data_cat, left_index=True, right_index=True).merge(test_data_num, left_index=True, right_index=True).to_pickle('/content/drive/MyDrive/kaggle/amex-default-predict/test_data_{}.pkl'.format(i))

    del test_data_time, test_data_num, test_data_cat, test_data_part
    gc.collect()


0 77052
1 77052
2 77052
3 77052
4 77052
